In [206]:
import pandas as pd
import numpy as np

In [207]:
##获取所需要的字段名称
columns_list=['客户代码','货物状态','交货方式','仓库名字','创建时间','宽CM','长CM','高CM','预报数量','收货数量']

In [208]:
##获取数据指定数据
data_detail=pd.read_csv(r'/Users/lijianwei/Desktop/明细.csv',usecols=columns_list)

In [209]:
data_detail.tail()

,客户代码,货物状态,交货方式,仓库名字,创建时间,宽CM,收货数量,长CM,预报数量,高CM
884026,G1928,在途,铁运,英国仓库,2020/3/20 17:53,10.0,0,28.0,26,10.0
884027,G1928,在途,铁运,英国仓库,2020/3/20 17:53,11.5,0,34.0,28,10.5
884028,G1928,在途,铁运,英国仓库,2020/3/20 17:53,11.5,0,32.5,30,10.0
884029,G1928,在途,铁运,英国仓库,2020/3/20 17:53,11.5,0,32.5,34,10.0
884030,G1928,在途,铁运,英国仓库,2020/3/20 17:53,11.5,0,32.5,42,10.0


In [210]:
##获取在途数据
print(data_detail['货物状态'].unique())

['在途' '收货中' '其他']


In [211]:
##排除货物状态为‘其他'的数据
data_detail=data_detail[data_detail['货物状态']!='其他']
print(data_detail['货物状态'].unique())

['在途' '收货中']


In [212]:
##新增字段，在途体积（单位立方米），在途数量
data_detail['在途数量']=data_detail['预报数量']-data_detail['收货数量']
data_detail['在途体积']=data_detail['在途数量']*data_detail['长CM']*data_detail['宽CM']*data_detail['高CM']/1000000

In [213]:
data_detail.head()

,客户代码,货物状态,交货方式,仓库名字,创建时间,宽CM,收货数量,长CM,预报数量,高CM,在途数量,在途体积
0,G666,在途,海运整柜,美东仓库,2020/4/3 11:43,68.0,0,102.0,1,12.0,1,0.083232
1,G666,在途,海运整柜,美东仓库,2020/4/3 11:43,68.0,0,102.0,1,12.0,1,0.083232
2,G666,在途,海运整柜,美东仓库,2020/4/3 11:43,68.0,0,102.0,1,12.0,1,0.083232
3,G666,在途,海运整柜,美东仓库,2020/4/3 11:43,68.0,0,102.0,1,12.0,1,0.083232
4,G666,在途,海运整柜,美东仓库,2020/4/3 11:43,68.0,0,102.0,1,12.0,1,0.083232


In [214]:
##获取货运时效表
ways_time=pd.read_excel(r'/Users/lijianwei/Desktop/各仓库不同货运分时时效.xlsx',header=0)

In [215]:
ways_time.head()

,仓库,货运方式,时效
0,法国仓库,海运散货,51
1,捷克仓库,海运散货,46
2,美东仓库,海运散货,53
3,美南仓库,海运散货,43
4,美西仓库,海运散货,36


In [216]:
##在data_detail中添加时效列
new_data=pd.merge(data_detail,ways_time,left_on=['交货方式','仓库名字'],right_on=['货运方式','仓库'],how='left')

In [217]:
new_data.head()

,客户代码,货物状态,交货方式,仓库名字,创建时间,宽CM,收货数量,长CM,预报数量,高CM,在途数量,在途体积,仓库,货运方式,时效
0,G666,在途,海运整柜,美东仓库,2020/4/3 11:43,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51.0
1,G666,在途,海运整柜,美东仓库,2020/4/3 11:43,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51.0
2,G666,在途,海运整柜,美东仓库,2020/4/3 11:43,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51.0
3,G666,在途,海运整柜,美东仓库,2020/4/3 11:43,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51.0
4,G666,在途,海运整柜,美东仓库,2020/4/3 11:43,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51.0


In [218]:
#删除空字段
new_data.dropna(subset=['时效'],inplace=True)
new_data['时效']=pd.Series.astype(new_data['时效'],np.int32)
new_data['时效'].dtype

dtype('int32')

In [219]:
##将时效列转为timedelta,将创建时间转为datetime
new_data['时效']=pd.to_timedelta(new_data['时效'],unit='d')
new_data['创建时间']=pd.to_datetime(new_data['创建时间'])

#计算预计到货日期
new_data['预计到货日期']=new_data['创建时间']+new_data['时效']

In [220]:
new_data.head()

,客户代码,货物状态,交货方式,仓库名字,创建时间,宽CM,收货数量,长CM,预报数量,高CM,在途数量,在途体积,仓库,货运方式,时效,预计到货日期
0,G666,在途,海运整柜,美东仓库,2020-04-03 11:43:00,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51 days,2020-05-24 11:43:00
1,G666,在途,海运整柜,美东仓库,2020-04-03 11:43:00,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51 days,2020-05-24 11:43:00
2,G666,在途,海运整柜,美东仓库,2020-04-03 11:43:00,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51 days,2020-05-24 11:43:00
3,G666,在途,海运整柜,美东仓库,2020-04-03 11:43:00,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51 days,2020-05-24 11:43:00
4,G666,在途,海运整柜,美东仓库,2020-04-03 11:43:00,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51 days,2020-05-24 11:43:00


In [221]:
#将预计到货日期设置为索引
new_data.set_index(['预计到货日期'],inplace=True)

In [222]:
#获取5月来货的数据
_5mGoods=new_data.loc['2020-5']

In [223]:
_5mGoods.head()

,客户代码,货物状态,交货方式,仓库名字,创建时间,宽CM,收货数量,长CM,预报数量,高CM,在途数量,在途体积,仓库,货运方式,时效
预计到货日期,,,,,,,,,,,,,,,
2020-05-24 11:43:00,G666,在途,海运整柜,美东仓库,2020-04-03 11:43:00,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51 days
2020-05-24 11:43:00,G666,在途,海运整柜,美东仓库,2020-04-03 11:43:00,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51 days
2020-05-24 11:43:00,G666,在途,海运整柜,美东仓库,2020-04-03 11:43:00,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51 days
2020-05-24 11:43:00,G666,在途,海运整柜,美东仓库,2020-04-03 11:43:00,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51 days
2020-05-24 11:43:00,G666,在途,海运整柜,美东仓库,2020-04-03 11:43:00,68.0,0,102.0,1,12.0,1,0.083232,美东仓库,海运整柜,51 days


In [224]:
##获取要统计的字段
new_5mGoods=pd.DataFrame(_5mGoods,columns=['仓库名字','客户代码','在途数量','在途体积'])

In [225]:
#进行分组
new_5mGoods=new_5mGoods.groupby(['仓库名字','客户代码']).sum()

In [226]:
new_5mGoods.head()

在途数量      在途体积
仓库名字  客户代码                 
意大利仓库 G1032   410  4.006763
      G1116  1497  2.891141
      G672    221  0.260257
      G836    397  7.549102
捷克仓库  105    1004  2.353032

In [227]:
#根据在途体积排序
new=pd.DataFrame()
for warehouse,data1 in new_5mGoods.groupby('仓库名字'):
    data1=data1.sort_values('在途体积',ascending=False)
    new=pd.concat([new,data1],axis=0)

In [228]:
new.head()

在途数量        在途体积
仓库名字  客户代码                    
意大利仓库 G836     397    7.549102
      G1032    410    4.006763
      G1116   1497    2.891141
      G672     221    0.260257
捷克仓库  200    23648  637.831105

In [229]:
#获取累加体积
new1=pd.Series()
for warehouse,data1 in new.groupby('仓库名字'):
    new1=new1.append(data1['在途体积'].cumsum())
new['累积体积']=new1

In [230]:
new.head()

在途数量        在途体积        累积体积
仓库名字  客户代码                                
意大利仓库 G836     397    7.549102    7.549102
      G1032    410    4.006763   11.555865
      G1116   1497    2.891141   14.447006
      G672     221    0.260257   14.707263
捷克仓库  200    23648  637.831105  637.831105

In [231]:
#获取占比前百分之八十的客户信息
new1=pd.Series()
for warehouse,data1 in new.groupby('仓库名字'):
    total_volume=data1['在途体积'].sum()
    new1=new1.append(data1['在途体积'].cumsum()/total_volume)
new['体积占比']=new1
new_80perc=new[new['体积占比']<=0.80]

In [232]:
new_80perc

在途数量          在途体积          累积体积      体积占比
仓库名字  客户代码                                                
意大利仓库 G836       397      7.549102      7.549102  0.513291
      G1032      410      4.006763     11.555865  0.785725
捷克仓库  200      23648    637.831105    637.831105  0.154614
      G975      5520    472.351402   1110.182507  0.269114
      G1032    23959    456.940814   1567.123321  0.379879
      G1065    13744    452.018524   2019.141845  0.489451
      G1921     1479    266.286036   2285.427881  0.554000
      G836     23624    261.626967   2547.054847  0.617420
      G635      1599    202.458984   2749.513831  0.666497
      G256     11897    192.888493   2942.402324  0.713254
      913258   10149    186.844857   3129.247181  0.758546
      159       4070    145.762980   3275.010161  0.793880
法国仓库  G1048    12297    549.827706    549.827706  0.641855
      G1921      590     65.320448    615.148153  0.718108
      G1641      545     62.507617    677.655771  0.791078
澳洲仓库  G1009    16854     78.264633     78.264633  0.453268
      G1253     6828     56.504125    134.768758  0.780510
美东仓库  G666    256714  10673.376287  10673.376287  0.598989
      23       19738   1889.728647  12563.104934  0.705040
      133       7876   1030.308924  13593.413858  0.762861
      G585      4082    382.670569  13976.084426  0.784336
美南仓库  G666      9250    687.826361    687.826361  0.557263
      133        882    212.285934    900.112295  0.729253
美西仓库  G666    101021   3815.355593   3815.355593  0.707550
      200       9023    219.613693   4034.969287  0.748277
      23        1662    177.350773   4212.320060  0.781166
英国仓库  G536     26762    648.160717    648.160717  0.213989
      G666     14179    454.360860   1102.521578  0.363996
      13       55429    315.227674   1417.749252  0.468068
      G1671     8060    275.810863   1693.560114  0.559126
      G658      3222    192.944577   1886.504692  0.622827
      G1852     1248    140.858288   2027.362980  0.669331
      G679     11214    123.818664   2151.181644  0.710209
      G1065     2602    123.082623   2274.264267  0.750845
      G256      2861     94.782473   2369.046739  0.782137
西班牙仓库 G1116      604      0.778077      0.778077  0.664444